In [1]:
# Add path candidates to sys.path to import utility / helper modules
import sys
sys.path.append('/workspace')

In [2]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [3]:
"""
Quick testing with existing data files

"""
from contextlib import redirect_stdout
from difflib import unified_diff
from io import StringIO
from pathlib import Path
from re import findall


# Run main() with data files and prints out differences between
# expected data and output
def run(main):
    # Lookup data file with directory name
    cwd = Path.cwd()
    data = cwd / f'{cwd.name}.dat'

    # If <working_directory_name>.dat is not found
    # then find any first data file in working directory
    if not data.exists():
        data = None
        for child in cwd.iterdir():
            if child.name.endswith('.dat'):
                data = child
                break

    # If there is no data file, do panic
    if data is None:
        raise FileNotFoundError('.dat file is not found')

    # Read data file and mock stdin with StringIO
    with data.open(mode='r', encoding='utf-8') as fp:
        data = fp.read()
        input = findall(r'^\[in\]([\d\w\s]*)\[out\]', data)[0]
        ifs = StringIO(input.strip())
        expected = findall(r'\[out\]([\s\S]*)$', data)[0]

    # Run program and with redirection of stdout to buffer
    with StringIO() as ofs, redirect_stdout(ofs):
        main(ifs.readline)
        result = ofs.getvalue()

    # Print differences
    sys.stdout.writelines(
        unified_diff(
            expected.lstrip().splitlines(keepends=True),
            result.lstrip().splitlines(keepends=True),
            fromfile='expected',
            tofile='result',
        )
    )

In [4]:
"""
Version 1:
- used methods: none

-----ooooo
----o-oooo
    ...
-ooooo----  comb(n+m-1, n-1)

o-----oooo
o----o-ooo
    ...
ooooo-----  comb(n+m-1, m-1)

total comb(n+m, n)
"""
from functools import lru_cache
from math import factorial


# Alternative to math.comb of Python version >= 3.8
@lru_cache(maxsize=100*100)
def comb(n, r):
    return factorial(n) // factorial(n - r) // factorial(r)


# Generate k-th morse signal containing :n '-' letter and :m 'o' letter
def solution(n, m, k):

    def solve(n, m, skip):
        if n == 0:
            return 'o' * m

        # :c is the number of cases where letter '-' comes first
        c = comb(n + m - 1, n - 1)

        # if :skip == :c, it should starts with letter 'o' so should use '<'
        if skip < c:
            return '-' + solve(n - 1, m, skip)

        return 'o' + solve(n, m - 1, skip - c)

    return solve(n, m, k - 1)


# Main I/O part
def main(rl):
    C = int(rl())
    for _ in range(C):
        n, m, k = map(int, rl().split())
        result = solution(n, m, k)
        print(result)


# Additional codes to simulate I/O
try:
    import IPython
except ImportError as _:
    # Submit env
    import sys
    main(sys.stdin.readline)
else:
    # IPython env
    run(main)

In [5]:
"""
Testing

"""
import cProfile
import traceback
import helpers as hlpr


f = hlpr.clock(hlpr.timeout(seconds=10)(solution))
test_cases = [
    ((2, 2, 4, ),
     'o--o'),
    ((4, 8, 13, ),
     '--o-ooo-oooo'),
    ((6, 4, 1, ),
     '------oooo'),
    ((100, 100, 10**9, ),
     '--------------------------------------------------'
     '--------------------------------------------oooooo'
     'oooooooo-oo-oooooooooooooooooooooooooooooooooooooo'
     'ooo-oooooooooooooooooooooo-ooooo-oooooooooooo-oooo'),
]

passed, failed = [], []
for idx, tc in enumerate(test_cases, start=1):
    case, expected = tc
    result = None
    try:
        result = f(*case)
        assert result == expected, \
            'wants {} but got {}'.format(hlpr.truncate(expected, 20), hlpr.truncate(result, 20))  # noqa: E501
    except Exception as err:
        fmt = 'Case {} FAIL: {}'
        print(fmt.format(idx, str(err)))
        traceback.print_exc()
        failed.append(idx)
    else:
        fmt = 'Case {} PASS'
        print(fmt.format(idx))
        passed.append(idx)

    print()

print()
if len(passed) == len(test_cases):
    print('All test(s) have passed')
else:
    fmt = '{} of {} test(s) failed: {}'
    print(fmt.format(len(failed), len(test_cases), ', '.join(map(str, failed))))  # noqa: E501

[0.00003680] 'solution(2, 2, 4)' → "'o--o'"
Case 1 PASS

[0.00017040] 'solution(4, 8, 13)' → "'--o-ooo-oooo'"
Case 2 PASS

[0.00003170] 'solution(6, 4, 1)' → "'------oooo'"
Case 3 PASS

[0.00078050] 'solution(100, 100, 1000000000)' → "'----------------------------------------------------------...
Case 4 PASS


All test(s) have passed
